In [1]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep
from kafka import KafkaConsumer
import pyspark

In [2]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_NAME')
kafka_topic_partition = os.getenv('KAFKA_TOPIC_NAME')

In [4]:
print(kafka_host)
print(kafka_topic)
print(kafka_topic_partition)

dataeng-kafka
apotek-dibimbing-ajinusa
apotek-dibimbing-ajinusa


In [5]:
# # from kafka import KafkaConsumer

# # To consume latest messages and auto-commit offsets
# consumer = KafkaConsumer(kafka_topic_partition,
#                          group_id='ajinusa-group',
#                          bootstrap_servers=[f'{kafka_host}:9092'])

In [6]:
# Consumer 1

from kafka import KafkaConsumer
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

# Mengonfigurasi Consumer untuk Kafka
consumer = KafkaConsumer(
    'apotek-dibimbing-ajinusa',  # Nama topik
    bootstrap_servers=f'{kafka_host}:9092',  # Kafka broker
    group_id='ajinusa-group',  # Nama Consumer Group
    auto_offset_reset='earliest',  # Mulai membaca dari offset pertama
    enable_auto_commit=True,  # Auto commit untuk offset
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))  # Deserialisasi pesan JSON
)


# Membaca pesan dan menyimpannya ke dalam list
data = []
for message in consumer:
    data.append(message.value)  # Tambahkan pesan ke list
    # print(f"Pesan diterima: {message.value}")
    # print(data)
    if len(data) >= 50:  # Ambil sample 50 pesan 
        break


# Membuat SparkSession
spark = SparkSession.builder \
    .appName("ajinusa-consumer") \
    .getOrCreate()


columns = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("buyer_name", StringType(), True),
    StructField("medication_name", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),
    StructField("total_price", IntegerType(), True),
    StructField("payment_method", StringType(), True),
    StructField("transaction_date", StringType(), True),
    StructField("transaction_time", StringType(), True),
])


# Konversi ke DataFrame
df_data = spark.createDataFrame(data, schema=columns)
df_data = df_data.withColumn("eom", last_day("transaction_date"))


# Tampilkan DataFrame
df_data.show()

print("===================== Total Sales by Payment Method (Consumer 1) =====================")
df_sales1 = df_data.groupBy("payment_method").agg(round(F.sum("total_price"),2).alias("total_sales"))
df_sales1 = df_sales1.orderBy("total_sales", ascending=False)
df_sales1.show()

# print("===================== Total Sales by Month =====================")
# df_sales = df_data.groupBy("eom").agg(round(F.sum("total_price"),2).alias("total_sales"))
# df_sales = df_sales.orderBy("eom", ascending=False).limit(10)
# df_sales.show()

exit()

+--------------------+-----------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|      transaction_id|       buyer_name|medication_name|quantity|unit_price|total_price|payment_method|transaction_date|transaction_time|       eom|
+--------------------+-----------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|b0148fca-38b8-497...|    Alicia Steele|    Paracetamol|       4|      2555|      10220|    Debit Card|      2025-01-15|        08:15:32|2025-01-31|
|7f8c63d3-872c-447...|    Sharon Hooper|     Antibiotik|       1|     11459|      11459|          Cash|      2025-01-12|        00:13:16|2025-01-31|
|49c9982c-73d2-4bc...|    Monica Massey|      Ibuprofen|       4|     15292|      61168|    Debit Card|      2025-01-17|        00:34:10|2025-01-31|
|a9038fbf-8a6e-4d1...|    Jennifer Pena|    Amoxicillin|       4|     16181|      64724|          Cash|   

In [7]:
# Consumer 2

from kafka import KafkaConsumer
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

# Mengonfigurasi Consumer untuk Kafka
consumer2 = KafkaConsumer(
    'apotek-dibimbing-ajinusa',  # Nama topik
    bootstrap_servers=f'{kafka_host}:9092',  # Kafka broker
    group_id='ajinusa-group',  # Nama Consumer Group
    auto_offset_reset='earliest',  # Mulai membaca dari offset pertama
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))  # Deserialisasi pesan JSON
)


# Membaca pesan dan menyimpannya ke dalam list
data2 = []
for message in consumer2:
    data2.append(message.value)  # Tambahkan pesan ke list
    if len(data2) >= 50:  # Ambil sample 50 pesan 
        break


# Membuat SparkSession 2
spark2 = SparkSession.builder \
    .appName("ajinusa-consumer2") \
    .getOrCreate()


columns = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("buyer_name", StringType(), True),
    StructField("medication_name", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),
    StructField("total_price", IntegerType(), True),
    StructField("payment_method", StringType(), True),
    StructField("transaction_date", StringType(), True),
    StructField("transaction_time", StringType(), True),
])


# Konversi ke DataFrame
df_data = spark2.createDataFrame(data2, schema=columns)
df_data = df_data.withColumn("eom", last_day("transaction_date"))


# Tampilkan DataFrame
df_data.show()

# print("===================== Total Sales by Payment Method =====================")
# df_sales1 = df_data.groupBy("payment_method").agg(round(F.sum("total_price"),2).alias("total_sales"))
# df_sales1 = df_sales1.orderBy("total_sales", ascending=False)
# df_sales1.show()

print("===================== Total Sales by Month (Consumer 2) =====================")
df_sales = df_data.groupBy("eom").agg(round(F.sum("total_price"),2).alias("total_sales"))
df_sales = df_sales.orderBy("eom", ascending=False).limit(10)
df_sales.show()

exit()

+--------------------+-----------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|      transaction_id|       buyer_name|medication_name|quantity|unit_price|total_price|payment_method|transaction_date|transaction_time|       eom|
+--------------------+-----------------+---------------+--------+----------+-----------+--------------+----------------+----------------+----------+
|b0148fca-38b8-497...|    Alicia Steele|    Paracetamol|       4|      2555|      10220|    Debit Card|      2025-01-15|        08:15:32|2025-01-31|
|7f8c63d3-872c-447...|    Sharon Hooper|     Antibiotik|       1|     11459|      11459|          Cash|      2025-01-12|        00:13:16|2025-01-31|
|49c9982c-73d2-4bc...|    Monica Massey|      Ibuprofen|       4|     15292|      61168|    Debit Card|      2025-01-17|        00:34:10|2025-01-31|
|a9038fbf-8a6e-4d1...|    Jennifer Pena|    Amoxicillin|       4|     16181|      64724|          Cash|   

In [5]:
# Consumer 3: coba ambil data dari partition tertentu

from kafka import TopicPartition
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

partition = 2

consumer3 = KafkaConsumer(group_id='ajinusa-group',
                         bootstrap_servers=[f'{kafka_host}:9092'],
                         value_deserializer=lambda m: json.loads(m.decode('ascii')),
                         # auto_offset_reset='earliest',
                         enable_auto_commit=False
                        )
consumer3.assign([TopicPartition(kafka_topic_partition, partition)])


# Membaca pesan dan menyimpannya ke dalam list
data3 = []
for message in consumer3:
    data3.append(message.value)  # Tambahkan pesan ke list
    print(data3)
    if len(data3) >= 5:  # Ambil sample 50 pesan 
        break

# Membuat SparkSession
spark3 = SparkSession.builder \
    .appName("ajinusa-consumer3") \
    .getOrCreate()

print(f"Spark version: {spark3.version}")


columns = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("buyer_name", StringType(), True),
    StructField("medication_name", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),
    StructField("total_price", IntegerType(), True),
    StructField("payment_method", StringType(), True),
    StructField("transaction_date", StringType(), True),
    StructField("transaction_time", StringType(), True),
])


# Konversi ke DataFrame
df_data = spark3.createDataFrame(data3, schema=columns)
df_data = df_data.withColumn("eom", last_day("transaction_date"))


# Tampilkan DataFrame
df_data.show()

print(f"===================== Consumer 3 : Partition {partition} =====================")

print("===================== Total Sales by Payment Method =====================")
df_sales1 = df_data.groupBy("payment_method").agg(round(F.sum("total_price"),2).alias("total_sales"))
df_sales1 = df_sales1.orderBy("total_sales", ascending=False)
df_sales1.show()

print("===================== Total Sales by Month =====================")
df_sales = df_data.groupBy("eom").agg(round(F.sum("total_price"),2).alias("total_sales"))
df_sales = df_sales.orderBy("eom", ascending=False).limit(10)
df_sales.show()

exit()


[{'transaction_id': '75dde4e4-3a16-4d7e-a4ab-91d44471be83', 'buyer_name': 'Joyce Coleman', 'medication_name': 'Amoxicillin', 'quantity': 3, 'unit_price': 7739, 'total_price': 23217, 'payment_method': 'QRIS', 'transaction_date': '2025-01-03', 'transaction_time': '09:16:30'}]
[{'transaction_id': '75dde4e4-3a16-4d7e-a4ab-91d44471be83', 'buyer_name': 'Joyce Coleman', 'medication_name': 'Amoxicillin', 'quantity': 3, 'unit_price': 7739, 'total_price': 23217, 'payment_method': 'QRIS', 'transaction_date': '2025-01-03', 'transaction_time': '09:16:30'}, {'transaction_id': '86180f8f-578e-4a2a-9e1a-bf6d97ef667d', 'buyer_name': 'Rebecca Torres', 'medication_name': 'Cough Syrup', 'quantity': 4, 'unit_price': 15315, 'total_price': 61260, 'payment_method': 'Cash', 'transaction_date': '2025-01-17', 'transaction_time': '15:31:16'}]
[{'transaction_id': '75dde4e4-3a16-4d7e-a4ab-91d44471be83', 'buyer_name': 'Joyce Coleman', 'medication_name': 'Amoxicillin', 'quantity': 3, 'unit_price': 7739, 'total_price':